# ``jenga`` Corruptions

This notebook illustrates some of the corruptions used in jenga

Most corruptions are operating on tabular data, for which different sampling mechanisms are used. 

Inspired by research on missing values, we consider three different sampling schemes for corrupting tabular data (we'll keep using the 'missingness' terminology here, even if we use other corruptions than discarding data):

* **MCAR** Missing completely at random
* **MAR** Missing at random - corruption is conditioned on other column
* **MNAR** Missing not at random - corruption is conditioned on values in column on which it is applied


In [ ]:
%load_ext autoreload
%autoreload 2

import random
import pandas as pd
import numpy as np

from jenga.corruptions.generic import MissingValues

In [ ]:
def new_df(N=20):
    return pd.DataFrame({
        'A':np.arange(0,N) / N, 
        'B': [random.choice(['a','b','c']) for _ in range(N)],
        'C': [random.choice(['foo','bah']) for _ in range(N)]
        })

## Missing Values

Below some examples on different row sampling strategies for corruptions with missing values

In [ ]:
from jenga.corruptions.generic import MissingValues

df = new_df()

df['B_MCAR'] = MissingValues(column='B', fraction=.5, missingness='MCAR').transform(df)['B']
df['B_MAR'] = MissingValues(column='B', fraction=.5, missingness='MAR').transform(df)['B']
df['B_MNAR'] = MissingValues(column='B', fraction=.5, missingness='MNAR').transform(df)['B']

df

If we sort according to the values in the column on which the corruption as applied, we see that the 'MNAR' condition works as it should

In [ ]:
df.sort_values('B')


## Swapping Values

Also works between numeric and non-numeric values, with different sampling schemes

In [ ]:
from jenga.corruptions.generic import SwappedValues
df = new_df()
df['C_swapped_MAR'] = SwappedValues(column='B', fraction=.5, sampling='MAR').transform(df)['C']
df['C_swapped_MCAR'] = SwappedValues(column='B', fraction=.5, sampling='MCAR').transform(df)['C']
df['A_swapped_MCAR'] = SwappedValues(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

## Messing up categorical Values

These corruptions permute the histogram of values

In [ ]:
from jenga.corruptions.generic import CategoricalShift
df = new_df()
df['C_permuted_MAR'] = CategoricalShift(column='B', fraction=.5, sampling='MAR').transform(df)['C']
df['C_permuted_MCAR'] = CategoricalShift(column='B', fraction=.5, sampling='MCAR').transform(df)['C']
df['A_permuted_MCAR'] = CategoricalShift(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

## Adding Gaussian Noise



In [ ]:
from jenga.corruptions.numerical import GaussianNoise
df = new_df()
df['A_GaussNoise_MAR'] = GaussianNoise(column='A', fraction=.5, sampling='MAR').transform(df)['A']
df['A_GaussNoise__MCAR'] = GaussianNoise(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

## Scaling numeric values


In [ ]:
from jenga.corruptions.numerical import Scaling
df = new_df()
df['A_Scaled_MAR'] = Scaling(column='A', fraction=.5, sampling='MAR').transform(df)['A']
df['A_Scaled_MCAR'] = Scaling(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df